# Normalizer

This is a component that normalize samples individually to unit norm using an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load a model.
- how to use a model to provide real-time transformations.

In [ ]:
%%writefile Transformer.py
import logging
from typing import List, Iterable, Union

import numpy as np
import pandas as pd
from platiagro import load_model, stat_dataset

logger = logging.getLogger(__name__)


class Transformer(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        logger.info("Initializing")

        # Load Estimator
        model = load_model(experiment_id=experiment_id)
        self.estimator = model["estimator"]
        
        # Load Columns
        metadata = stat_dataset(name=dataset)
        self.columns = metadata["columns"]

        logger.info("Init complete!")

    def class_names(self):
        return self.columns

    def transform_input(self, X: np.ndarray, feature_names: Iterable[str]) -> Union[np.ndarray, List, str, bytes]:
        """Returns a transformation on input data.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.

        Returns:
            numpy.array: Array-like with transformations.
        """
        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        # Perform Transformation
        return self.estimator.transform(df.to_numpy())

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%sh
MODEL_NAME=Transformer
API_TYPE=REST
SERVICE_TYPE=TRANSFORMER
PERSISTENCE=0
LOG_LEVEL=DEBUG
PARAMETERS='[{"type":"STRING","name":"dataset","value":"boston"},{"type":"STRING","name":"target","value":"medv"},{"type":"STRING","name":"experiment_id","value":"e6c07a4e-4902-43c3-b02f-efe8f051cb0c"}]'

seldon-core-microservice $MODEL_NAME $API_TYPE \
    --service-type $SERVICE_TYPE \
    --persistence $PERSISTENCE \
    --parameters $PARAMETERS \
    --log-level $LOG_LEVEL > log.txt 2>&1 &

until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    printf '.'
    sleep 1
done

## Make transformations

In [ ]:
!curl localhost:5000/transform-input -d 'json={"data":{"names":["crim", "zn", "indus", "chas", "nox", "rm", "age", "dis", "rad", "tax", "ptratio", "black", "lstat"], "ndarray":[[0.00632, 18.0, 2.31, 0, 0.5379999999999999, 6.575, 65.2, 4.09, 1, 296, 15.3, 396.9, 4.98]]}}'

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill